In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import seaborn as sns

data_path = pathlib.Path().home()/'group'/'project'/'results'/'beamforming'\
/'static'/'beamform_signals'/'220309_0to5cm_84to90_8192sample_static_bf_signals.npz'

#data_path = pathlib.Path().home()/'group'/'project'/'results'/'beamforming'\
#/'time_dependent'/'beamform_signals'/'220307_0to5cm_84to90_8192_gradb_corrected.npz'
def interpolate(missing_inds, image):
    
    skipped_pixels = []
    
    for i in range(missing_inds.shape[0]):
        
        neighbor_x = []
        neighbor_y = []
        
        missing_y = missing_inds[i, 0]
        missing_x = missing_inds[i, 1]
        
        if missing_x + 1 < image.shape[1]:
            neighbor_x.append(missing_x + 1)
        if missing_x - 1 >= 0:
            neighbor_x.append(missing_x - 1)
        if missing_y + 1 < image.shape[0]:
            neighbor_y.append(missing_y + 1)
        if missing_y - 1 >= 0:
            neighbor_y.append(missing_y - 1)
            
        neighbor_x.append(missing_x)
        neighbor_y.append(missing_y)
            
        neighbor_grid_x, neighbor_grid_y = np.meshgrid(neighbor_x, neighbor_y)
        
        freq_list = []
        
        for j, pair in enumerate(zip(neighbor_grid_x.flatten(), neighbor_grid_y.flatten())):
            #print(pair, missing_inds[:, 0], missing_inds[:, 1])
            if not np.any(np.logical_and(missing_inds[:, 0] == pair[0], missing_inds[:, 1] == pair[1])):
                freq_list.append(image[pair[1], pair[0]])
                #print(freq_list)
                #input()
                
        if len(freq_list) == 0:
            skipped_pixels.append([missing_x, missing_y])
        else:
            image[missing_y, missing_x] = np.mean(freq_list)
    skipped_pixels = np.array(skipped_pixels)
    
    if skipped_pixels.shape[0] == 0:
        return image
    else:
        interpolate(skipped_pixels, image)

In [ ]:
data = np.load(data_path)
for i, key in enumerate(data): print(key)

In [ ]:
print(data['signals'].shape)

In [ ]:
print(data['radius'])

In [ ]:
r_grid, pa_grid = np.meshgrid(
    np.sort(np.unique(data['radius'])),
    np.sort(np.unique(data['pitch'])),
                              )

print(r_grid.flatten())
signals_sort = np.zeros((r_grid.size, data['signals'].shape[-1]), np.complex128)
signals = data['signals'][:]
for i, pair in enumerate(zip(r_grid.flatten(), pa_grid.flatten())):
    
    #print(i)
    try:
        ind = np.argwhere(np.logical_and(
            data['radius'] == pair[0],
            data['pitch'] == pair[1]
        )).squeeze()
        signals_sort[i, :] = signals[ind, :]
        #print(i, ind)
    except:
        pass

In [ ]:
print(pa_grid.flatten()[0:100], data['pitch'][0:100])

In [ ]:
power_grid = np.mean(abs(signals_sort) ** 2, axis=-1).reshape((r_grid.shape))

In [ ]:
ind_max = np.argmax(abs(np.fft.fft(signals_sort, axis=-1)), axis=-1)


carrier_grid = abs(np.diag(np.fft.fft(signals_sort, axis=-1)[:, ind_max] / 8192)).reshape((r_grid.shape))

In [ ]:
plt.imshow(power_grid , aspect='auto', )
plt.colorbar()

In [ ]:
plt.imshow(carrier_grid ** 2  / (60*50), aspect='auto', )
plt.colorbar()

In [ ]:
np.savez(
    pathlib.Path().home()/'group'/'project'/'results'\
    /'beamforming'/'static'/'220329_static_bf_carrier_power_map',
    carrier_power=(carrier_grid ** 2) / (60 * 50),
    pitch_angle_grid=pa_grid,
    radius_grid=r_grid
)

In [ ]:
np.savez(
    pathlib.Path().home()/'group'/'project'/'results'\
    /'beamforming'/'static'/'220328_static_bf_power_map',
    power=power_grid,
    pitch_angle_grid=pa_grid,
    radius_grid=r_grid
)

# save the power grid image, if desired

In [ ]:
missing_inds = np.argwhere(power_grid==0)

power_grid_interpolate = interpolate(missing_inds, power_grid)

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('plasma', as_cmap=True)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(
    power_grid_interpolate, 
    aspect='auto',
    extent=(0, 0.05, 90, 84),
    cmap=cmap,
)
cbar = fig.colorbar(img, label=r'$|V_{rms}^2|$')

ax.set_xlabel('Radius (m)')
ax.set_ylabel('Pitch Angle (deg)')

plt.tight_layout()
save_path = pathlib.Path.home()/'group'/'project'/'plots'/'analysis'/'beamforming'/'power_maps'
name = '220318_static_beamforming_power_map'
#plt.savefig(save_path/name)

In [ ]:
print((10 * 50 * 60 * 200e6 * 1.38e-23) / 8192)

In [ ]:
max_inds = np.argmax(abs(np.fft.fft(signals_sort)), axis=-1)

In [ ]:


peak_grid = np.diag((abs(np.fft.fft(signals_sort) / 8192) ** 2)[:, max_inds])

In [ ]:
peak_grid.shape

In [ ]:
plt.imshow(peak_grid.reshape((r_grid.shape)), aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
threshold = 10 ** (5.0/5) * (10 * 50 * 60 * 200e6 * 1.38e-23) / 8192

In [ ]:
inds_above_T = np.argwhere(peak_grid>= threshold).squeeze()

In [ ]:
radius_above_T = r_grid.flatten()[inds_above_T]
pitch_above_T = pa_grid.flatten()[inds_above_T]

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.imshow(
    peak_grid.reshape((r_grid.shape)),
    aspect='auto',
    interpolation='none',
    extent=(0.0, 0.05, 90, 84)
)

ax.plot(radius_above_T, pitch_above_T, '.')

# select the boundary above threshold points

In [ ]:
unique_radius = np.sort(np.unique(radius_above_T))
boundary_pitch = np.zeros(unique_radius.size)
for i, iradius in enumerate(unique_radius):
    
    where_radius = np.argwhere(radius_above_T == iradius).squeeze()
    
    boundary_pitch[i] = np.min(pitch_above_T[where_radius])
    

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.imshow(
    peak_grid.reshape((r_grid.shape)),
    aspect='auto',
    interpolation='none',
    extent=(0.0, 0.05, 90, 84)
)

ax.plot(unique_radius, boundary_pitch, '.')

# save the coordinates for the boundary of the BF detection

In [ ]:
save_path = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/\
'power_map_detection_boundary'
name = '220309_static_bf_10db_threshold_boundary'

np.savez(save_path/name, radius=unique_radius, pitch=boundary_pitch)

# create the mf boundary

In [ ]:
print(signals.shape)

In [ ]:
norm = 1 / abs(np.sqrt(10 * 50 * 60 * 200e6 * 1.38e-23 * (signals_sort.conjugate() * signals_sort).sum(-1)))

In [ ]:
norm.shape

In [ ]:
scores = norm * abs((signals_sort.conjugate() * signals_sort).sum(-1))

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
img = ax.imshow(
    scores.reshape((r_grid.shape)),
    aspect='auto',
    interpolation='none',
    extent=(0.0, 0.05, 90, 84)
)
cbar = fig.colorbar(img)


In [ ]:
threshold = 2.

inds_above_T = np.argwhere(scores>= threshold).squeeze()
radius_above_T = r_grid.flatten()[inds_above_T]
pitch_above_T = pa_grid.flatten()[inds_above_T]

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.imshow(
    scores.reshape((r_grid.shape)),
    aspect='auto',
    interpolation='none',
    extent=(0.0, 0.05, 90, 84)
)

ax.plot(radius_above_T, pitch_above_T, '.')

In [ ]:
unique_radius = np.sort(np.unique(radius_above_T))
boundary_pitch = np.zeros(unique_radius.size)
for i, iradius in enumerate(unique_radius):
    
    where_radius = np.argwhere(radius_above_T == iradius).squeeze()
    
    boundary_pitch[i] = np.min(pitch_above_T[where_radius])

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.imshow(
    scores.reshape((r_grid.shape)),
    aspect='auto',
    interpolation='none',
    extent=(0.0, 0.05, 90, 84)
)

ax.plot(unique_radius, boundary_pitch, '.', color='red')

In [ ]:
save_path = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/\
'power_map_detection_boundary'
name = '220308_mf_score2_threshold_boundary'

np.savez(save_path/name, radius=unique_radius, pitch=boundary_pitch)

In [ ]:
noise = np.random.multivariate_normal(
    [0, 0], np.eye(2) * 5 * 50 * 60 * 1.38e-23 * 200e6 / 8192, 8192
)
noise = noise[:, 0] + 1j * noise[:, 1]

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(noise) ** 2)
ax.plot(abs(np.fft.fftshift(np.fft.fft(signals_sort[4000, :]) / 8192)) ** 2)